In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from tqdm import tqdm
from PIL import Image
import pickle
import numpy as np
from sklearn.neural_network import MLPRegressor

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train/training_images.csv')

In [3]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/images/'+train['file_name'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:02<00:00, 61.31it/s]


In [4]:
y=train['infected'].values
y = to_categorical(y)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [8]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [9]:
y_pred = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 131 samples, validate on 33 samples
Epoch 1/10
131/131 [==============================] - ETA: 2s - loss: 0.6989 - acc: 0.437 - ETA: 1s - loss: 0.7900 - acc: 0.437 - ETA: 0s - loss: 0.7607 - acc: 0.479 - ETA: 0s - loss: 0.7481 - acc: 0.492 - 1s 11ms/step - loss: 0.7472 - acc: 0.4885 - val_loss: 0.6589 - val_acc: 0.7273
Epoch 2/10
131/131 [==============================] - ETA: 0s - loss: 0.7193 - acc: 0.406 - ETA: 0s - loss: 0.6696 - acc: 0.562 - ETA: 0s - loss: 0.6941 - acc: 0.520 - ETA: 0s - loss: 0.6843 - acc: 0.554 - 1s 4ms/step - loss: 0.6834 - acc: 0.5573 - val_loss: 0.6213 - val_acc: 0.7273
Epoch 3/10
131/131 [==============================] - ETA: 0s - loss: 0.6714 - acc: 0.562 - ETA: 0s - loss: 0.6645 - acc: 0.578 - ETA: 0s - loss: 0.6791 - acc: 0.562 - ETA: 0s - loss: 0.6773 - acc: 0.585 - 1s 4ms/step - loss: 0.6756 - acc: 0.5954 - val_loss: 0.6524 - val_acc: 0.7273
Epoch 4/10
131/131 [==============================] - 

In [10]:
test = pd.read_csv('test/test_images.csv')

In [11]:
test.head()

,file_name,infected
0,23.infected%20wound.jpg,NaN
1,23.shutterstock_361288121-189209.jpg,NaN
2,23.wound-infected.png,NaN
3,24.7403-skin_infection_from_wound-1296x728-sli...,NaN
4,24.c1066-woundinfection.jpg,NaN


In [12]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/images/'+test['file_name'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test_array = np.array(test_image)

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 73.97it/s]


In [13]:
prediction = model.predict_classes(test_array)

In [14]:
print(prediction)

[1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1]


In [15]:
test['infected'] = prediction

In [16]:
print(test)

                                            file_name  infected
0                             23.infected%20wound.jpg         1
1                23.shutterstock_361288121-189209.jpg         1
2                               23.wound-infected.png         1
3   24.7403-skin_infection_from_wound-1296x728-sli...         0
4                         24.c1066-woundinfection.jpg         1
5         24.fibrinous%20to%20granulating%20wound.jpg         0
6   25.3-s2.0-B9781437727869000126-f012-006-978143...         1
7                         25.c1066-woundinfection.jpg         1
8   25.Fight-or-flight-hormone-could-impact-wound-...         1
9   26.7403-skin_infection_from_wound-1296x728-sli...         0
10                       26.getty_rf_blood_finger.jpg         1
11                                    26.IMG_0925.jpg         1
12  27.Longstanding-nonhealing-infected-wound-to-t...         1
13                               27.maxresdefault.jpg         0
14        28.amputation_site_infected_an

In [17]:
with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)